---

# Очистка данных

***Мусор на входе модели $\to$ мусор на выходе.***

Ни одна модель машинного обучения не выдаст осмысленных результатов, если вы предоставите ей сырые данные. После формирования выборки данных их необходимо очистить.

In [70]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#
from tqdm import tqdm 

pd.set_option("display.max_columns", 300)

PATH_TRAIN = './data/train_sber.csv'
PATH_TEST = './data/test.csv'
PATH_MACRO = './data/macro.csv'

In [25]:
df_train = pd.read_csv(PATH_TRAIN)
df_makro = pd.read_csv(PATH_MACRO)
df_train_y = df_train['price_doc']
df_train.drop(columns='price_doc', axis=1, inplace=True)

In [105]:
def moustache(s):
    # Возращает значения усов нормального распределения
    q75,q25 = np.percentile(s,[75,25])
    iqr = q75 - q25
    return q25 - 1.5*iqr, q75 + 1.5*iqr

def data_cleanup_table(df):
    # Возвращает таблицу для EDA с анализом признаков
    
    # Создание таблицы
    columns = ['Признак', 'Тип_данных', 'Пропущенных_значений', 
           'Выбросов', 'Ошибочных_значений', 'Управление', 'Комментарий']
    table = pd.DataFrame(columns=columns)
    table['Признак'] = df.columns
    print('Создание таблицы...')
    #
    for sign in df.columns:
        table.loc[table['Признак'] == sign, 'Тип_данных'] = df[sign].dtype
    
    
    # Анализ пропущенных значений
    for sign in tqdm(df.columns):
        table.loc[table['Признак'] == sign, 'Пропущенных_значений'] = df[sign].isna().sum()
    print('Анализ пропущенных значений выполнен')
    
    # Анализ выбрасов
    for sign in tqdm(df.columns):
        if df[sign].dtype == 'object':
            table.loc[table['Признак'] == sign, 'Выбросов'] = -1
        else:
            tmp = moustache(df[sign]) # возращает границы усов (а, б)
            table.loc[table['Признак'] == sign, 'Выбросов'] = df[(df[sign] < tmp[0]) |
                                                                            (df[sign] > tmp[1])][sign].count()
    print('Анализ выбросов выполнен')
    
    return table

In [106]:
table = data_cleanup_table(df_train)
table.head()

Создание таблицы...


100%|█████████████████████████████████████████| 291/291 [00:00<00:00, 911.69it/s]


Анализ пропущенных значений выполнен


100%|█████████████████████████████████████████| 291/291 [00:02<00:00, 116.58it/s]

Анализ выбросов выполнен


,Признак,Тип_данных,Пропущенных_значений,Выбросов,Ошибочных_значений,Управление,Комментарий
0,id,int64,0,0,NaN,NaN,NaN
1,timestamp,object,0,-1,NaN,NaN,NaN
2,full_sq,int64,0,963,NaN,NaN,NaN
3,life_sq,float64,6383,0,NaN,NaN,NaN
4,floor,float64,167,0,NaN,NaN,NaN


In [102]:
df.loc[:,df.dtypes == 'object']

,timestamp,product_type,sub_area,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,water_1line,big_road1_1line,railroad_1line,ecology
0,2011-08-20,Investment,Bibirevo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2011-08-23,Investment,Nagatinskij Zaton,yes,no,no,no,no,no,no,no,no,no,no,no,excellent
2,2011-08-27,Investment,Tekstil'shhiki,no,no,no,no,yes,no,no,no,no,no,no,no,poor
3,2011-09-01,Investment,Mitino,no,no,no,no,no,no,no,no,no,no,no,no,good
4,2011-09-05,Investment,Basmannoe,no,no,no,no,yes,yes,no,no,no,no,no,yes,excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,2015-06-30,Investment,Otradnoe,no,no,yes,no,yes,no,no,no,no,no,no,no,good
30467,2015-06-30,Investment,Tverskoe,yes,no,no,no,yes,yes,no,no,yes,no,no,no,poor
30468,2015-06-30,OwnerOccupier,Poselenie Vnukovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
30469,2015-06-30,Investment,Obruchevskoe,no,no,no,no,yes,no,no,no,no,no,no,no,satisfactory


In [96]:
(df_train.isna().mean().round(4)*100)[df_train.isna().mean().round(4)*100 > 0]

life_sq                                  20.95
floor                                     0.55
max_floor                                31.41
material                                 31.41
build_year                               44.65
num_room                                 31.41
kitch_sq                                 31.41
state                                    44.50
preschool_quota                          21.95
school_quota                             21.94
hospital_beds_raion                      47.39
raion_build_count_with_material_info     16.38
build_count_block                        16.38
build_count_wood                         16.38
build_count_frame                        16.38
build_count_brick                        16.38
build_count_monolith                     16.38
build_count_panel                        16.38
build_count_foam                         16.38
build_count_slag                         16.38
build_count_mix                          16.38
raion_build_c